In [26]:
library(mvtnorm)
library(matlib)

In [27]:
p <- 3
n.W1 <- 150
n.W2 <- 150
n <- n.W1 + n.W2

In [28]:
get_covariance_matrix <- function(n) {
    A <- matrix(runif(n^2) * 2 - 1, ncol = n) 
    Sigma <- t(A) %*% A
    
    return(Sigma)
}
get_means_vector <- function(n, a, b) {
    mu <- runif(n, a, b)
    
    return(mu)
}

## Well separable data

In [29]:
Sigma.W1 <- get_covariance_matrix(p)
Sigma.W2 <- Sigma.W1
Sigma.W1

1.862422624,0.94265821,0.007369189
0.942658213,0.52130597,-0.061378646
0.007369189,-0.06137865,0.108230849


In [30]:
mu.W1 <- get_means_vector(p, 3, 4)
mu.W2 <- get_means_vector(p, 1, 1.5)
mu.W1
mu.W2

[1] 3.726551 3.324112 3.204446

[1] 1.364601 1.234509 1.030329

In [31]:
X.W1 <- data.frame(rmvnorm(n.W1, mu.W1, Sigma.W1), class = matrix(1, n.W1, 1))
X.W2 <- data.frame(rmvnorm(n.W2, mu.W2, Sigma.W2), class = matrix(2, n.W2, 1))

In [32]:
q1 <- 0.4
q2 <- 1 - q1

n.test <- 100

### Create test data

In [33]:
X.test <- rbind(data.frame(rmvnorm(as.integer(q1 * n.test), mu.W1, Sigma.W1), 
                           class = matrix(1, as.integer(q1 * n.test), 1)),
                data.frame(rmvnorm(as.integer(q2 * n.test), mu.W2, Sigma.W2), 
                           class = matrix(2, as.integer(q2 * n.test), 1)))
X.test <- X.test[sample(nrow(X.test)), ]
X.test[0:5, ]

,X1,X2,X3,class
,<dbl>,<dbl>,<dbl>,<dbl>
92,-0.7240756,0.1260534,1.1569803,2
79,0.2143908,0.9956213,0.5161142,2
49,1.7690091,1.7542950,0.6450259,2
25,4.2805036,3.5158146,3.0654554,1
26,1.9856671,2.1509164,3.5655482,1


### Discriminant function

In [34]:
alpha <- inv(Sigma.W1) %*% (mu.W1 - mu.W2)
alpha

-222.0019
438.8968
284.1055


In [35]:
xi1 <- sum(alpha * mu.W1)
xi2 <- sum(alpha * mu.W2)

c <- (xi1 + xi2) / 2
c

[1] 1036.821

In [36]:
get_sigma_z_squared <- function(alpha, Sigma, p) {
    sigma_z.squared = 0
    for (m in (1:p)) {
        for (j in (1:p)) {
            sigma_z.squared = sigma_z.squared + alpha[m] * Sigma[m, j] * alpha[j]
        }
    } 
    return(sigma_z.squared)
}

In [37]:
sigma_z.squared <- get_sigma_z_squared(alpha, Sigma.W1, p)
sigma_z.squared

[1] 1010.441

In [38]:
dist.mhl <- (xi1 - xi2)^2 / sigma_z.squared
dist.mhl

[1] 1010.441

In [39]:
predict_class <- function(alpha, c, X, q1, q2, xi1, xi2) {
    predicted <- matrix(0, dim(X)[1], 1)
    
    for (i in 1:dim(X)[1]) {
        alpha_X <- sum(alpha * X[i,])
        predicted[i] = ifelse(alpha_X >= (xi1 + xi2) * 0.5 + log(q2 / q1), 1, 2)
    }
    
    return(predicted)
}

In [40]:
X.predict <- predict_class(alpha, c, X.test[,-4], q1, q2, xi1, xi2)

In [41]:
length(X.predict[X.predict == 1,])

[1] 40

In [42]:
length(X.predict[X.predict == 2,])

[1] 60

In [43]:
X_1.predict <- predict_class(alpha, c, X.W1[,-4], q1, q2, xi1, xi2)

### Confusion table

In [47]:
get_conf_table <- function(data.predicted, deta.real) {
    conf_table.test <- data.frame(matrix(0, 2, 2))
    colnames(conf_table.test) <- c("1_pred", "2_pred")
    rownames(conf_table.test) <- c("1_real", "2_real")
    
    for (i in 1:2) {
        for (j in 1:2) {
            conf_table.test[i, j] <- sum(data.predicted[which(deta.real == i),] == j)
        }
    }
    
    return(conf_table.test)
    
}

In [45]:
get_conf_table(X.predict, X.test$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,40,0
2_real,0,60


In [46]:
sum(X.predict == X.test$class)

[1] 100

## Badly separable data

In [48]:
Sigma.W1.bad <- get_covariance_matrix(p)
Sigma.W2.bad <- Sigma.W1.bad
Sigma.W1.bad

1.6584829,-0.3333887,1.1790574
-0.3333887,1.6829961,-0.7560935
1.1790574,-0.7560935,1.1013027


In [50]:
mu.W1.bad <- get_means_vector(p, 1.2, 2)
mu.W2.bad <- get_means_vector(p, 1, 1.5)
mu.W1.bad
mu.W2.bad

[1] 1.366229 1.514021 1.272475

[1] 1.268416 1.125645 1.082894

In [51]:
X.W1.bad <- data.frame(rmvnorm(n.W1, mu.W1.bad, Sigma.W1.bad), 
                       class = matrix(1, n.W1, 1))
X.W2.bad <- data.frame(rmvnorm(n.W2, mu.W2.bad, Sigma.W2.bad), 
                       class = matrix(2, n.W2, 1))

X.test.bad <- rbind(data.frame(rmvnorm(as.integer(q1 * n.test), mu.W1.bad, Sigma.W1.bad), 
                           class = matrix(1, as.integer(q1 * n.test), 1)),
                    data.frame(rmvnorm(as.integer(q2 * n.test), mu.W2.bad, Sigma.W2.bad), 
                           class = matrix(2, as.integer(q2 * n.test), 1)))

X.test.bad <- X.test.bad[sample(nrow(X.test.bad)), ]
X.test.bad[0:5, ]

,X1,X2,X3,class
,<dbl>,<dbl>,<dbl>,<dbl>
28,-0.5505544,1.25402254,0.4912645,1
6,3.5299695,-0.19634002,3.1834345,1
75,2.6995519,0.03006373,2.0105399,2
27,1.4125120,-0.68459986,2.5798690,1
36,2.1712297,3.39131885,0.3676728,1


In [52]:
alpha.bad <- inv(Sigma.W1.bad) %*% (mu.W1.bad - mu.W2.bad)
alpha.bad

-1.574923
1.089722
2.606402


In [53]:
xi1.bad <- sum(alpha.bad * mu.W1.bad)
xi2.bad <- sum(alpha.bad * mu.W2.bad)

c.bad <- (xi1.bad + xi2.bad) / 2
c.bad

[1] 2.433089

In [54]:
sigma_z.squared.bad <- get_sigma_z_squared(alpha.bad, Sigma.W1.bad, p)
sigma_z.squared.bad

[1] 0.7632969

In [55]:
dist.mhl.bad <- (xi1.bad - xi2.bad)^2 / sigma_z.squared.bad
dist.mhl.bad

[1] 0.7632969

In [56]:
X.predict.bad <- predict_class(alpha.bad, c.bad, 
                               X.test.bad[,-4], 
                               q1, q2, 
                               xi1.bad, xi2.bad)

In [57]:
get_conf_table(X.predict.bad, X.test.bad$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,20,20
2_real,10,50
